Web Scarping World Meters Data

In [1]:
import pandas as pd

In [3]:
#Import Relevant Libraries
import pandas as pd
from bs4 import BeautifulSoup as BS
import requests
from splinter import Browser
from Driver import chromedriver
import os

Connect to Chromedriver Via Outside file

In [3]:
executable_path = {'executable_path': 'chromedriver' }
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
#URL to be scraped
url = 'https://www.worldometers.info/coronavirus/country/us/'
#Retrieve page with request module
browser.visit(url)
html = browser.html
# make soup and get all tables
soup = BS(html, 'html.parser')

In [5]:
table = soup.find('table') 
print(table)

<table class="table table-bordered table-hover table-responsive usa_table_countries" id="usa_table_countries_today" style="width:100%">
<thead>
<tr>
<th width="100">USA<br/>State</th>
<th width="20">Total<br/>Cases</th>
<th width="30">New<br/>Cases</th>
<th width="30">Total<br/>Deaths</th>
<th width="30">New<br/>Deaths</th>
<th width="30">Active<br/>Cases</th>
<th width="30">Tot Cases/<br/>1M pop</th>
<th width="30">Deaths/<br/>1M pop</th>
<th width="30">Total<br/>Tests</th>
<th width="30">Tests/<br/>
<nobr>1M pop</nobr>
</th>
<th width="30">Source</th>
</tr>
</thead>
<tbody>
<tr class="total_row_usa">
<td style="text-align:left;">USA Total</td>
<td>1,236,880</td>
<td>+24,045</td>
<td>72,222</td>
<td>+2,301</td>
<td>964,974</td>
<td>3,737</td>
<td>218</td>
<td>7,697,091</td>
<td>23,254</td>
<td> </td>
</tr>
<tr style="">
<td style="font-weight: bold; font-size:15px; text-align:left;">
New York </td>
<td style="font-weight: bold; text-align:right">330,139 </td>
<td style="font-weight: b

Parse through soup to find table headers

In [6]:
table_header = table.find_all('th')

In [7]:
table_header

[<th width="100">USA<br/>State</th>,
 <th width="20">Total<br/>Cases</th>,
 <th width="30">New<br/>Cases</th>,
 <th width="30">Total<br/>Deaths</th>,
 <th width="30">New<br/>Deaths</th>,
 <th width="30">Active<br/>Cases</th>,
 <th width="30">Tot Cases/<br/>1M pop</th>,
 <th width="30">Deaths/<br/>1M pop</th>,
 <th width="30">Total<br/>Tests</th>,
 <th width="30">Tests/<br/>
 <nobr>1M pop</nobr>
 </th>,
 <th width="30">Source</th>]

In [8]:
#Create list of headers
header = [i.text.strip() for i in table_header]

Parse Through Beautiful Soup to Find All Table Data

In [9]:
table_rows = table.find_all('tr')

In [10]:
rows = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [i.text.strip() for i in td]
    rows.append(row)

Clean Data

In [11]:
table_data = pd.DataFrame(rows, columns = header)
table_data

,USAState,TotalCases,NewCases,TotalDeaths,NewDeaths,ActiveCases,Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop,Source
0,None,None,None,None,None,None,None,None,None,None,None
1,USA Total,"1,236,880","+24,045","72,222","+2,301","964,974","3,737",218,"7,697,091","23,254",
2,New York,"330,139","+2,765","25,204",+260,"251,204","16,828","1,285","1,028,899","52,445",[1] [2] [3] [4] [5] [6] [7] [8] [9] [10]
3,New Jersey,"131,606","+2,261","8,273",+322,"122,062","14,817",931,"275,900","31,063",[view by county] [1]
4,Massachusetts,"70,271","+1,184","4,212",+122,"57,941","10,288",617,"333,349","48,805",[1] [2]
...,...,...,...,...,...,...,...,...,...,...,...
60,Federal Prisons,"2,425",+85,41,+1,"1,692",,,,,[1]
61,Grand Princess Ship,103,,3,,100,,,,,[1]
62,Wuhan Repatriated,3,,,,3,,,3,,
63,Diamond Princess Ship,46,,,,46,,,46,,


In [12]:
worldmetersUS = table_data.drop('Source', axis=1)

In [13]:
worldmetersUS = worldmetersUS.drop(0, axis=0)

In [14]:
path = './Resources'
worldmetersUS.to_csv(os.path.join(path, 'worldmetersUS.csv'))

PermissionError: [Errno 13] Permission denied: './Resources\\worldmetersUS.csv'

In [23]:
worldmetersUS = pd.read_csv('./Resources/worldmetersUS.csv')

In [24]:
US_states_df = US_States['USAState']

In [25]:
US_states_df = pd.DataFrame(US_states_df)

In [26]:
US_states_df

,USAState
0,USA Total
1,New York
2,New Jersey
3,Massachusetts
4,Illinois
...,...
59,Federal Prisons
60,Grand Princess Ship
61,Wuhan Repatriated
62,Diamond Princess Ship


### Retrieve Latitudes and Longitudes for each US State

In [27]:
from opencage.geocoder import OpenCageGeocode
import pandas as pd

In [28]:
#Enter Open Cage key and import it into function
key = "03aad42af4334ab88a21c8aaa4cbcc02"
geocoder = OpenCageGeocode(key)

In [21]:
#Build loop to request lat and long from Geocoder

#List to append info into
lat = []
lng = []
query = []

#loops through data frame takes county and state and request info
for index, row in US_states_df.iterrows():
    if row['USAState'] == 'PR':
        query = "u'" + ", " + row['State'] + " ,PRI'"
        results = geocoder.geocode(query)
        #Saves lat and long into list
        lat.append(results[0]['geometry']['lat']) 
        lng.append(results[0]['geometry']['lng'])
    else:
        query = "u'" + ", " + row['USAState'] + " ,US'"
        results = geocoder.geocode(query)
        #Saves lat and long into list
        lat.append(results[0]['geometry']['lat']) 
        lng.append(results[0]['geometry']['lng'])

In [29]:
worldmetersUS['Latitude'] = lat

In [31]:
worldmetersUS['Longitude'] = lng

In [32]:
worldmetersUS

,Unnamed: 0,USAState,TotalCases,NewCases,TotalDeaths,NewDeaths,ActiveCases,Total_Cases,Deaths/1M pop,TotalTests,Tests\n1M pop,Latitude,Longitude
0,1,USA Total,"1,218,638","5,803","70,727",806.0,"959,133","3,682",214,"7,535,605","22,766",37.291140,-120.505736
1,2,New York,"327,374",NaN,"25,174",230.0,"248,855","16,687","1,283","1,007,310","51,345",41.676259,-73.896615
2,3,New Jersey,"129,345",NaN,"7,951",NaN,"120,123","14,563",895,"275,900","31,063",40.700948,-74.173630
3,4,Massachusetts,"69,087",NaN,"4,090",NaN,"56,879","10,115",599,"324,268","47,476",42.350724,-71.324371
4,5,Illinois,"63,840",NaN,"2,662",NaN,"60,533","4,979",208,"333,147","25,983",41.780119,-87.643841
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,60,Federal Prisons,"2,340",NaN,40,NaN,"1,608",NaN,NaN,NaN,NaN,37.842690,-90.504733
60,61,Grand Princess Ship,103,NaN,3,NaN,100,NaN,NaN,NaN,NaN,41.141023,-94.984766
61,62,Wuhan Repatriated,3,NaN,NaN,NaN,3,NaN,NaN,3,NaN,20.881910,-156.520641
62,63,Diamond Princess Ship,46,NaN,NaN,NaN,46,NaN,NaN,46,NaN,39.766121,-105.606489


In [34]:
path = './Resources'
worldmetersUS.to_csv(os.path.join(path, 'worldmetersUS1.csv'))